In [6]:
import numpy as np
V = (1/2)**2*np.pi
rho = 2700
m = V*rho
I = 1/2*m*(1/2)**2
print(I)

alpha = 1

tau = I*alpha

265.0718801466388


In [ ]:
import os
import shutil

files = ['compute_torque.exp', 'compute_torque.lib', 'compute_torque.pyd', 'inverse_kinematics_cpp.exp', 'inverse_kinematics_cpp.lib', 'inverse_kinematics_cpp.pyd']

# Remove the files from the current directory
for file in files:
    file_path = os.path.join(dir, file)
    if os.path.exists(file_path):
        try:
            os.remove(file_path)
            print(f"Removed: {file_path}")
        except PermissionError:
            print(f"Could not remove (permission denied): {file_path}")
    else:
        print(f"File not found: {file_path}")


# Copy the files from the build directory
relative_path = os.path.join(dir, 'robot_dynamic_module', 'build', 'Release')
for file in files:
    src = os.path.join(relative_path, file)
    dst = os.path.join(dir, file)
    shutil.copyfile(src, dst)
    print(f"Copied: {src} -> {dst}")

File not found: d:\FreeCAD\Mod\FreeCAD-robotic-addon\compute_torque.exp
File not found: d:\FreeCAD\Mod\FreeCAD-robotic-addon\compute_torque.lib
File not found: d:\FreeCAD\Mod\FreeCAD-robotic-addon\compute_torque.pyd
File not found: d:\FreeCAD\Mod\FreeCAD-robotic-addon\inverse_kinematics_cpp.exp
File not found: d:\FreeCAD\Mod\FreeCAD-robotic-addon\inverse_kinematics_cpp.lib
File not found: d:\FreeCAD\Mod\FreeCAD-robotic-addon\inverse_kinematics_cpp.pyd


FileNotFoundError: [Errno 2] No such file or directory: 'robot_dynamic_module\\build\\Release\\compute_torque.exp'